In [33]:
import pandas as pd
from pandas_datareader import data
from datetime import datetime
from dateutil.relativedelta import relativedelta
import requests
from enum import Enum
import pprint
import math
import json

pp = pprint.PrettyPrinter(indent=1)

ALPHA_VANTAGE_API_KEY = '111111111111'

SEMICON_TICKERS = [
    'INTC',
    'MU',
    'TXN',
    'QCOM',
    'WOLF',
    'AMBA',
    'AMAT',
    'NXPI',
    'SWKS',
    'NVDA',
    'TSM',
    'AVGO',
    'ASX',
    'STM',
]


# Notes

## Key indicators

1. Gross profit margin = (revenue - cost) / revenue \* 100
2. EBITDA margin = EBITDA / revenue \* 100
3. Expenditure as % of revenue = expenditure / revenue
4. Net current asset value = (total liabilities - current assets) / number of outstanding shares
5. Net working capital
6. Benjamin Graham valuation formula (3% risk free rate)
7. Earnings Power Value
8. Fair Value PE = Basic PE x [1 + (1 - Business Risk)] x [1 + (1 - Financial Risk)] x [1 + (1 - Earnings Visibility)]

- Earnings growth rate
- Dividend yield
- Business risk
- Financial risk
- Earnings visibility

9. Reverse discounted cash flow (GIC dun like?)


In [37]:
class AlphaVantageFundamentalFunctions(Enum):
    INCOME_STATEMENT = 'INCOME_STATEMENT'
    BALANCE_SHEET = 'BALANCE_SHEET'
    CASH_FLOW = 'CASH_FLOW'
    COMPANY_OVERVIEW = 'OVERVIEW'
    EARNINGS = 'EARNINGS'  # not frequently used
    LISTING_AND_DELISTING_STATUS = 'LISTING_STATUS'  # not frequently used
    EARNINGS_CALENDAR = 'EARNINGS_CALENDAR'  # not frequently used
    IPO_CALENDAR = 'IPO_CALENDAR'  # not frequently used


def getTicker(companyName):
    url = f'https://www.alphavantage.co/query?function=SYMBOL_SEARCH&keywords={companyName}&apikey={ALPHA_VANTAGE_API_KEY}'
    response = requests.get(url=url)
    data = response.json()
    return [x.get('1. symbol') for x in data.get('bestMatches', [])]


def getFundamentalData(function: AlphaVantageFundamentalFunctions, ticker: str):
    url = f'https://www.alphavantage.co/query?function={function.value}&symbol={ticker}&apikey={ALPHA_VANTAGE_API_KEY}'
    r = requests.get(url)
    data = r.json()

    return data


# 1. Gross profit margin = (revenue - cost) / revenue * 100
def grossMargin(annualReport):
    revenue = int(annualReport.get('totalRevenue', '0'))
    profit = int(annualReport.get('profit', '0'))
    costs = [
        'costOfRevenue',
        # 'costofGoodsAndServicesSold',
        # 'operatingExpenses',
        # 'researchAndDevelopment',
        # 'sellingGeneralAndAdministrative'
    ]
    totalCosts = sum([int(annualReport.get(cost, '0')) for cost in costs])
    return (revenue - totalCosts) / revenue


# 2. EBITDA margin = EBITDA / revenue * 100
def EBITDAMargin(companyOverview):
    ebitda = int(companyOverview.get('EBITDA', '0'))
    revenue = int(companyOverview.get('RevenueTTM', '0'))
    return ebitda / revenue


# 3. Expenditure as % of revenue = expenditure / revenue
def expenditureOverRevenue(annualReport):
    revenue = int(annualReport.get('totalRevenue', '0'))
    expenses = [
        'operatingExpenses',
        'interestExpense',
        'incomeTaxExpense',
        'interestAndDebtExpense',
    ]
    totalExpenses = sum([int(annualReport.get(cost, '0'))
                        for cost in expenses])
    return totalExpenses / revenue


# 4. Net current asset value = (total liabilities - current assets) / number of outstanding shares
def netCurrentAssetValue(balanceSheet):
    liabilities = int(balanceSheet.get('totalLiabilities', '0'))
    assets = int(balanceSheet.get('totalAssets', '0'))
    outstandingShares = int(balanceSheet.get(
        'commonStockSharesOutstanding', '0'))
    return (liabilities - assets) / outstandingShares


# 5. Net working capital ratio = current assets / current liabilities
def netWorkingCapital(balanceSheet):
    liabilities = int(balanceSheet.get('totalLiabilities', '0'))
    assets = int(balanceSheet.get('totalAssets', '0'))
    return assets / liabilities


# 6. Benjamin Graham valuation formula (3% risk free rate)
def grahamNumber(companyOverview):
    bookValue = float(companyOverview.get('BookValue', '0'))
    eps = float(companyOverview.get('EPS', '0'))
    return math.sqrt(22.5 * eps * bookValue)


# 7. Earnings Power Value


# 8. Fair Value PE = Basic PE x [1 + (1 - Business Risk)] x [1 + (1 - Financial Risk)] x [1 + (1 - Earnings Visibility)]
#   - Earnings growth rate
#   - Dividend yield
#   - Business risk
#   - Financial risk
#   - Earnings visibility


# 9. Reverse discounted cash flow (GIC dun like?)


In [35]:
companyOverview = getFundamentalData(
    AlphaVantageFundamentalFunctions.COMPANY_OVERVIEW, 'AAPL')
annualReport = getFundamentalData(
    AlphaVantageFundamentalFunctions.INCOME_STATEMENT, 'AAPL').get('annualReports', [[]])[0]
balanceSheet = getFundamentalData(
    AlphaVantageFundamentalFunctions.BALANCE_SHEET, 'AAPL').get('annualReports', [[]])[0]
cashFlow = getFundamentalData(
    AlphaVantageFundamentalFunctions.CASH_FLOW, 'AAPL').get('annualReports', [[]])[0]


In [38]:
print(grossMargin(annualReport))
print(expenditureOverRevenue(annualReport))
print(netCurrentAssetValue(balanceSheet))
print(netWorkingCapital(balanceSheet))
print(EBITDAMargin(companyOverview))
print(grahamNumber(companyOverview))


0.35305034529093654
0.33841514146160623
0.17540999856817155
-3.8406782677999214
1.2191294562227348
23.986730706788702


In [12]:
pp.pprint(annualReport)
for k in annualReport.keys():
    if 'expense' in k.lower():
        print(k)


{'comprehensiveIncomeNetOfTax': '95249000000',
 'costOfRevenue': '234954000000',
 'costofGoodsAndServicesSold': '212981000000',
 'depreciation': '9500000000',
 'depreciationAndAmortization': '11284000000',
 'ebit': '111852000000',
 'ebitda': '123136000000',
 'fiscalDateEnding': '2021-09-30',
 'grossProfit': '152836000000',
 'incomeBeforeTax': '109207000000',
 'incomeTaxExpense': '14527000000',
 'interestAndDebtExpense': '2645000000',
 'interestExpense': '2645000000',
 'interestIncome': '2843000000',
 'investmentIncomeNet': '2843000000',
 'netIncome': '94680000000',
 'netIncomeFromContinuingOperations': '94680000000',
 'netInterestIncome': '-2645000000',
 'nonInterestIncome': '365817000000',
 'operatingExpenses': '43887000000',
 'operatingIncome': '108949000000',
 'otherNonOperatingIncome': '60000000',
 'reportedCurrency': 'USD',
 'researchAndDevelopment': '21914000000',
 'sellingGeneralAndAdministrative': '21973000000',
 'totalRevenue': '363172000000'}
operatingExpenses
interestExpense

In [11]:
pp.pprint(companyOverview)


{'200DayMovingAverage': '158.28',
 '50DayMovingAverage': '144.36',
 '52WeekHigh': '182.44',
 '52WeekLow': '129.04',
 'Address': 'ONE INFINITE LOOP, CUPERTINO, CA, US',
 'AnalystTargetPrice': '184.89',
 'AssetType': 'Common Stock',
 'Beta': '1.195',
 'BookValue': '4.158',
 'CIK': '320193',
 'Country': 'USA',
 'Currency': 'USD',
 'Description': 'Apple Inc. is an American multinational technology company '
                'that specializes in consumer electronics, computer software, '
                "and online services. Apple is the world's largest technology "
                'company by revenue (totalling $274.5 billion in 2020) and, '
                "since January 2021, the world's most valuable company. As of "
                "2021, Apple is the world's fourth-largest PC vendor by unit "
                'sales, and fourth-largest smartphone manufacturer. It is one '
                'of the Big Five American information technology companies, '
                'along with Amazon, Go